In [19]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [48]:
product_url = 'https://www.amazon.in/dp/B09RMQYHLH/ref=QAHzEditorial_en_IN_1?pf_rd_r=4EYV7CT7G3YK573MVN9C&pf_rd_p=826008c2-23c9-4704-9075-a5b1da8b89d3&pf_rd_m=A1VBAL9TL5WCBF&pf_rd_s=merchandised-search-7&pf_rd_t=&pf_rd_i=1389401031&ie=UTF8&ref_=CLP_MH8_BSAffordable_1'

In [49]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OSX 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko)Chrome/71.0.3578.98 Safari/537.36", 
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8"
}


def get_response(url: str) -> BeautifulSoup:
    '''A function to return a BeautifulSoup object
    '''
    res = requests.get(product_url, headers=HEADERS)
    return BeautifulSoup(res.content, 'html.parser')


def collect_data(soup: BeautifulSoup) -> pd.DataFrame():
    # collect title
    title = soup.select('#productTitle')[0].text.strip()

    # Collect specifications
    eles = soup.select('#feature-bullets li span')
    info = [ele.text.strip() for ele in eles]

    keys_raw = soup.select('#prodDetails > div > div:nth-child(1) > div:nth-child(1) > div')[0].select('th')
    keys = [key.text.strip() for key in keys_raw]

    values_raw = soup.select('#prodDetails > div > div:nth-child(1) > div:nth-child(1) > div')[0].select('td')
    values = [values.text.strip() for values in values_raw]

    other_keys = [x.text.strip() for x in soup.select('#productDetails_db_sections')[0].select('th')]
    other_values = [x.text.strip() for x in soup.select('#productDetails_db_sections')[0].select('td')]

    keys.extend(other_keys)
    values.extend(other_values)

    # Current Price
    price = soup.select('#corePriceDisplay_desktop_feature_div > div.a-section.a-spacing-none.aok-align-center > span.a-price.aok-align-center.reinventPricePriceToPayMargin.priceToPay > span:nth-child(2) > span.a-price-whole')[0].text


    reviews_url = 'https://amazon.in' + soup.select('#reviews-medley-footer > div.a-row.a-spacing-medium > a')[0]['href']


    keys = ['product name', 'info', 'current price', 'reviews'] + keys
    values= [title, '\n'.join(info), price, reviews_url] + values

    info_df = pd.DataFrame({
        'keys': keys,
        'values': values
    })

    return info_df

In [50]:
my_headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OSX 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko)Chrome/71.0.3578.98 Safari/537.36", 
          "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8"}

In [51]:
res = requests.get(product_url, headers=my_headers)
res

<Response [200]>

In [52]:
soup = get_response(product_url)

In [54]:
soup.select('p')

[<p class="a-last">Sorry, we just need to make sure you're not a robot. For best results, please make sure your browser is accepting cookies.</p>]

In [47]:
soup.select("#productDetails_techSpec_section_1")

[]

In [41]:
df = collect_data(soup)
df

IndexError: list index out of range

In [8]:
print(df[df['keys'] == 'reviews']['values'].values[0])

https://amazon.in/Airdopes-141-Playtime-Resistance-Bluetooth/product-reviews/B09N3ZNHTY/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews


In [9]:
#customer_review-R358NYWUQLR163

In [10]:
url = df[df['keys'] == 'reviews']['values'].values[0]

In [11]:
url

'https://amazon.in/Airdopes-141-Playtime-Resistance-Bluetooth/product-reviews/B09N3ZNHTY/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'

In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlencode
import csv

In [13]:
urls = [
    'https://amazon.in/Airdopes-141-Playtime-Resistance-Bluetooth/product-reviews/B09N3ZNHTY/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'
    'https://www.amazon.in/Airdopes-141-Playtime-Resistance-Bluetooth/product-reviews/B09N3ZNHTY/ref=cm_cr_getr_d_paging_btm_next_3?ie=UTF8&reviewerType=all_reviews&pageNumber=3'
]

In [14]:
names = []
reviews = []
data_string = ""
url = 'https://amazon.in/Airdopes-141-Playtime-Resistance-Bluetooth/product-reviews/B09N3ZNHTY/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'

from tqdm import tqdm

for i in tqdm(range(1, 100)):
    response = requests.get(url + f'?pageNumber={i}')
    if response.status_code != 200:
        break
    soup = BeautifulSoup(response.text, 'html.parser')

    for item in soup.findAll("a", {"data-hook": "review-title"}):
        data_string = data_string + item.get_text()
        names.append(data_string)
        data_string = ""  
    
    for item in soup.find_all("span", {"data-hook": "review-body"}):
        data_string = data_string + item.get_text()
        reviews.append(data_string)
        data_string = ""
    
    print(len(names), len(reviews))

  1%|▍                                           | 1/99 [00:01<03:02,  1.86s/it]

10 10


  2%|▉                                           | 2/99 [00:03<02:53,  1.79s/it]

20 20


  3%|█▎                                          | 3/99 [00:05<02:49,  1.77s/it]

...

  5%|██▏                                         | 5/99 [00:08<02:45,  1.76s/it]

50 50


  6%|██▋                                         | 6/99 [00:10<02:42,  1.75s/it]

60 60


  6%|██▋                                         | 6/99 [00:11<03:04,  1.99s/it]


In [15]:
# Create the dictionary.
reviews_dict = {'title': names, 'review': reviews}
# Print the lengths of each list.  
print(len(names), len(reviews))
# 72 50

60 60


In [16]:
# Create a new dataframe.
df = pd.DataFrame.from_dict(reviews_dict)
df.head()

,title,review
0,\ngreat product under low price range\n,"\nPros:1. The level of volume it gets to, is j..."
1,\nBEST For music! Totally worth it Edit: Not w...,"\nPros:1. The level of volume it gets to, is j..."
2,\n1 year urage review\n,\nSimply...This the best TWS in this budget se...
3,\nBeast in budget!\n,"\nBought it in January,posting it in August. S..."
4,\nLove the product 👌 ❤️\n,\nI love the product the noise cancelation is ...
